In [1]:
from repalette.lightning.datamodules import AdversarialDataModule
from repalette.lightning.systems import PreTrainSystem, AdversarialSystem
from repalette.constants import S3_PRETRAINED_MODEL_CHECKPOINT_PATH, LIGHTNING_LOGS_DIR

In [2]:
!source "../.env"

In [3]:
pretrain_system = PreTrainSystem.load_from_checkpoint("/home/dan/Projects/repalette/data/model-checkpoints/final/epoch=4-step=494599.ckpt")

In [4]:
adversarial_datamodule = AdversarialDataModule(multiplier=16, size=0.01, batch_size=8)

In [5]:
import pytorch_lightning as pl

In [6]:
from repalette.lightning.callbacks import LogTripletRecoloringToTensorboard
from pytorch_lightning.loggers import TensorBoardLogger
import uuid

log_recoloring_to_tensorboard = LogTripletRecoloringToTensorboard(batches=3)
logger = TensorBoardLogger(
    LIGHTNING_LOGS_DIR,
    name="final_train",
    log_graph=False,
    default_hp_metric=False,
    )

In [7]:
trainer = pl.Trainer(gpus=-1, max_epochs=3,
                     logger=logger,
                     benchmark=True,
                     callbacks=[
                         log_recoloring_to_tensorboard,
                     ]
                    )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [8]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.metrics.regression import MeanSquaredError
from torch import nn as nn

from repalette.constants import (
    DEFAULT_ADVERSARIAL_LR,
    DEFAULT_ADVERSARIAL_BETA_1,
    DEFAULT_ADVERSARIAL_BETA_2,
    DEFAULT_ADVERSARIAL_WEIGHT_DECAY,
    DEFAULT_ADVERSARIAL_LAMBDA_MSE_LOSS,
)
from repalette.models import PaletteNet, Discriminator
from repalette.utils.transforms import Scaler

In [9]:
adversarial_system = AdversarialSystem(generator=pretrain_system.generator)

In [ ]:
trainer.fit(adversarial_system, adversarial_datamodule)


  | Name          | Type          | Params
------------------------------------------------
0 | generator     | PaletteNet    | 13.9 M
1 | discriminator | Discriminator | 2.8 M 
2 | MSE           | MSELoss       | 0     
------------------------------------------------
16.7 M    Trainable params
0         Non-trainable params
16.7 M    Total params


In [11]:
LIGHTNING_LOGS_DIR

'/home/dan/Projects/repalette/data/lightning-logs'